Arshak Parsa

In [ ]:
import numpy as np
import pandas as pd

# Task 1 (Data Preparation and Building a Base Decision Tree)

## Part A (Loading and Preprocessing)

In [ ]:
df = pd.read_csv("data/dataset.csv")

## Part B (Training Decision Tree)

# Task 2 (Impact of model Complexity and Tree Pruning)

## Part A (Training a Tree without Depth Limit)

## Part B (Checking Different Depths)

## Part C (Cost Complexity Pruning)

## Final Results